## 빅데이터분석기사 작업형 2

이직여부 판단 데이터
- 데이터 설명 : 이직여부 판단 데이터 (target: 1: 이직 , 0 : 이직 x)
- x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv
- y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv
- x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv
- x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_test.csv
- 데이터 출처 :https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists (참고, 데이터 수정)

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv")

display(x_train.head())
display(y_train.head())

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,25298,city_138,0.836,Male,No relevent experience,Full time course,High School,NaN,5,100-500,Pvt Ltd,1,45
1,4241,city_160,0.920,Male,No relevent experience,Full time course,High School,NaN,5,NaN,NaN,1,17
2,24086,city_57,0.866,Male,No relevent experience,no_enrollment,Graduate,STEM,10,NaN,NaN,1,50
3,26773,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,>4,135
4,32325,city_143,0.740,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,17


,enrollee_id,target
0,25298,0.0
1,4241,1.0
2,24086,0.0
3,26773,0.0
4,32325,1.0


In [2]:
#결측치파악
print(x_train.isna().sum())
print(x_test.isna().sum())

enrollee_id                  0
city                         0
city_development_index       0
gender                    2917
relevent_experience          0
enrolled_university        257
education_level            315
major_discipline          1866
experience                  37
company_size              3852
company_type              3981
last_new_job               273
training_hours               0
dtype: int64
enrollee_id                  0
city                         0
city_development_index       0
gender                    1591
relevent_experience          0
enrolled_university        129
education_level            145
major_discipline           947
experience                  28
company_size              2086
company_type              2159
last_new_job               150
training_hours               0
dtype: int64


In [3]:
#결측치처리
x_train['gender'] = x_train['gender'].fillna(0)
x_train['enrolled_university'] = x_train['enrolled_university'].fillna(0)
x_train['education_level'] = x_train['education_level'].fillna(0)
x_train['major_discipline'] = x_train['major_discipline'].fillna(0)
x_train['experience'] = x_train['experience'].fillna(0)
x_train['company_size'] = x_train['company_size'].fillna(0)
x_train['company_type'] = x_train['company_type'].fillna(0)
x_train['last_new_job'] = x_train['last_new_job'].fillna(0)

x_test['gender'] = x_test['gender'].fillna(0)
x_test['enrolled_university'] = x_test['enrolled_university'].fillna(0)
x_test['education_level'] = x_test['education_level'].fillna(0)
x_test['major_discipline'] = x_test['major_discipline'].fillna(0)
x_test['experience'] = x_test['experience'].fillna(0)
x_test['company_size'] = x_test['company_size'].fillna(0)
x_test['company_type'] = x_test['company_type'].fillna(0)
x_test['last_new_job'] = x_test['last_new_job'].fillna(0)

In [4]:
#불필요한 컬럼제거
x_test_enrollee_id = x_test['enrollee_id']
x_train = x_train.drop(columns = ['enrollee_id','city'], axis = 1)
x_test = x_test.drop(columns = ['enrollee_id','city'], axis = 1)

In [5]:
# 범주형(object), 수치형(number) 변수 파악
obj = x_train.select_dtypes(include = 'object').columns
num = x_train.select_dtypes(include = 'number').columns
print(obj)
print(num) 

Index(['gender', 'relevent_experience', 'enrolled_university',
       'education_level', 'major_discipline', 'experience', 'company_size',
       'company_type', 'last_new_job'],
      dtype='object')
Index(['city_development_index', 'training_hours'], dtype='object')


In [6]:
# 범주형 변수(object) 전처리
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

In [7]:
# 수치형 변수(number) 전처리
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [8]:
# 모델링을 위해 데이터분리(data split)
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train['target'], test_size = 0.3, random_state = 1234)

In [9]:
# 모델링
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PRED = model.predict_proba(X_TEST)[:,1]

In [10]:
# 평가
from sklearn.metrics import roc_auc_score
print(roc_auc_score(Y_TEST, Y_TEST_PRED))

0.7922313682168454


In [11]:
#최종제출
y_test_pred = model.predict_proba(x_test)[:,1]
result = pd.DataFrame(y_test_pred)
print(result)
final = pd.concat([x_test_enrollee_id, result], axis = 1).rename(columns = {0:'target'})
print(final)
final.to_csv('123456.csv', index= False)

         0
0     0.81
1     0.60
2     0.53
3     0.49
4     0.24
...    ...
6701  0.12
6702  0.36
6703  0.22
6704  0.40
6705  0.48

[6706 rows x 1 columns]
      enrollee_id  target
0            7129    0.81
1           31037    0.60
2           22179    0.53
3           29724    0.49
4           17977    0.24
...           ...     ...
6701         3601    0.12
6702         2745    0.36
6703        18520    0.22
6704        10067    0.40
6705         8203    0.48

[6706 rows x 2 columns]


In [12]:
#제출확인
check = pd.read_csv('123456.csv')
print(check)

      enrollee_id  target
0            7129    0.81
1           31037    0.60
2           22179    0.53
3           29724    0.49
4           17977    0.24
...           ...     ...
6701         3601    0.12
6702         2745    0.36
6703        18520    0.22
6704        10067    0.40
6705         8203    0.48

[6706 rows x 2 columns]
